In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

In [ ]:
pipeline_options = PipelineOptions(['--direct_num_workers', '2'])

In [ ]:
with beam.Pipeline(options=pipeline_options) as pipeline:
  lines = (
      pipeline
      | beam.Create([
          'To be, or not to be: that is the question: ',
          "Whether 'tis nobler in the mind to suffer ",
          'The slings and arrows of outrageous fortune, ',
          'Or to take arms against a sea of troubles, ',
      ]))

In [ ]:
import re
inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Dataset' >> beam.Create([
          'To be, or not to be: that is the question: ',
          "Whether 'tis nobler in the mind to suffer ",
          'The slings and arrows of outrageous fortune, ',
          'Or to take arms against a sea of troubles, ',
      ])
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

In [ ]:
with beam.Pipeline(options=pipeline_options) as pipeline:
  (
      pipeline
      | 'Dataset' >> beam.Create([
          'To be or not to be: that is the question',
          "Whether tis nobler in the mind to suffer",
          'The slings and arrows of outrageous fortune',
          'Or to take arms against a sea of troubles, ',
      ])
      | 'Find words' >> beam.Map(lambda line: line.lower().split())
      | 'Pair words with 1' >> beam.FlatMap(lambda words: [(word, words) for word in words])
      | 'Pair words with 2' >> beam.FlatMap(lambda t: [(t[0] + '_' + word, 1) for word in t[1]])
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'Format results' >> beam.Filter(lambda word_count: int(word_count[1]) > 2)
      | 'Print words' >> beam.Map(print)
      #| 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )